<a href="https://colab.research.google.com/github/anileshwar12/DiseasePrediction/blob/main/disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Disease_symptom_and_patient_profile_dataset.csv to Disease_symptom_and_patient_profile_dataset.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier


In [ ]:
df = pd.read_csv('Disease_symptom_and_patient_profile_dataset.csv')

print(df.head())
print(df.columns)


       Disease Fever Cough Fatigue Difficulty Breathing  Age  Gender  \
0    Influenza   Yes    No     Yes                  Yes   19  Female   
1  Common Cold    No   Yes     Yes                   No   25  Female   
2       Eczema    No   Yes     Yes                   No   25  Female   
3       Asthma   Yes   Yes      No                  Yes   25    Male   
4       Asthma   Yes   Yes      No                  Yes   25    Male   

  Blood Pressure Cholesterol Level Outcome Variable  
0            Low            Normal         Positive  
1         Normal            Normal         Negative  
2         Normal            Normal         Negative  
3         Normal            Normal         Positive  
4         Normal            Normal         Positive  
Index(['Disease', 'Fever', 'Cough', 'Fatigue', 'Difficulty Breathing', 'Age',
       'Gender', 'Blood Pressure', 'Cholesterol Level', 'Outcome Variable'],
      dtype='object')


In [ ]:
df = df.dropna().reset_index(drop=True)

df.replace({'Yes': 1, 'No': 0}, inplace=True)

gender_encoder = LabelEncoder()
df['Gender'] = gender_encoder.fit_transform(df['Gender'])

disease_encoder = LabelEncoder()
df['Disease'] = disease_encoder.fit_transform(df['Disease'])

bp_map = {'Low': 0, 'Normal': 1, 'High': 2}
chol_map = {'Low': 0, 'Normal': 1, 'High': 2}
outcome_map = {'Negative': 0, 'Positive': 1}

df['Blood Pressure'] = df['Blood Pressure'].map(bp_map)
df['Cholesterol Level'] = df['Cholesterol Level'].map(chol_map)
df['Outcome Variable'] = df['Outcome Variable'].map(outcome_map)


/tmp/ipython-input-5-1950363480.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'Yes': 1, 'No': 0}, inplace=True)


In [ ]:
X = df.drop('Disease', axis=1)
y = df['Disease']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
probs = model.predict_proba(X_test)

top4_indices = np.argsort(probs, axis=1)[:, -4:][:, ::-1]
top4_percentages = np.take_along_axis(probs, top4_indices, axis=1)

results = []

for i in range(len(top4_indices)):
    sample_result = []
    for j in range(4):
        disease_name = disease_encoder.classes_[top4_indices[i][j]]
        probability = round(top4_percentages[i][j] * 100, 1)
        sample_result.append({"name": disease_name, "probability": probability})
    results.append(sample_result)


from pprint import pprint
pprint(results[:5])


[[{'name': 'Hyperglycemia', 'probability': np.float64(67.0)},
  {'name': 'Osteoarthritis', 'probability': np.float64(23.0)},
  {'name': 'Eczema', 'probability': np.float64(3.0)},
  {'name': 'Schizophrenia', 'probability': np.float64(2.0)}],
 [{'name': 'Hypothyroidism', 'probability': np.float64(37.8)},
  {'name': 'Kidney Cancer', 'probability': np.float64(36.2)},
  {'name': 'Schizophrenia', 'probability': np.float64(5.0)},
  {'name': 'Epilepsy', 'probability': np.float64(5.0)}],
 [{'name': 'Cholecystitis', 'probability': np.float64(65.0)},
  {'name': 'Bladder Cancer', 'probability': np.float64(8.0)},
  {'name': 'Hyperthyroidism', 'probability': np.float64(6.3)},
  {'name': 'Asthma', 'probability': np.float64(4.7)}],
 [{'name': 'Hemophilia', 'probability': np.float64(21.0)},
  {'name': 'Allergic Rhinitis', 'probability': np.float64(11.0)},
  {'name': 'Lung Cancer', 'probability': np.float64(8.4)},
  {'name': 'Lyme Disease', 'probability': np.float64(8.0)}],
 [{'name': 'Anxiety Disorders

In [ ]:
import joblib

joblib.dump(model, 'disease_model.pkl')
joblib.dump(disease_encoder, 'disease_encoder.pkl')
joblib.dump(gender_encoder, 'gender_encoder.pkl')


['gender_encoder.pkl']

In [ ]:
from google.colab import files
files.download('disease_model.pkl')
files.download('disease_encoder.pkl')
files.download('gender_encoder.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -q tensorflow


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)
y_ohe = ohe.fit_transform(y.values.reshape(-1, 1))

model_nn = keras.Sequential([
    layers.Input(shape=(X.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(y_ohe.shape[1], activation='softmax')
])

model_nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_nn.fit(X, y_ohe, epochs=30, batch_size=32)


Epoch 1/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 77ms/step - accuracy: 0.0174 - loss: 6.1972
Epoch 2/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0496 - loss: 4.6217  
Epoch 3/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0348 - loss: 4.5173     
Epoch 4/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0500 - loss: 4.4057 
Epoch 5/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0341 - loss: 4.4515 
Epoch 6/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0265 - loss: 4.4430     
Epoch 7/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0630 - loss: 4.3409 
Epoch 8/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0284 - loss: 4.4151     
Epoch 9/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0543 - loss: 4.3402 
Epoch 10/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0567 - loss: 4.3411 
Epoch 11/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0661 - loss: 4.3693 
Epoch 12/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_nn)
tflite_model = converter.convert()

with open('disease_model.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpy22qzmtt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 116), dtype=tf.float32, name=None)
Captures:
  136088334835472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136088334832016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136088334837584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136088334834512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136088334837776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136088334835664: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
import json

label_map = {i: str(disease_encoder.classes_[i]) for i in range(len(disease_encoder.classes_))}

with open('label_map.json', 'w') as f:
    json.dump(label_map, f)


In [ ]:
from google.colab import files
files.download('disease_model.tflite')
files.download('label_map.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>